# Multiple Alleic Association Test Module

## Input/Output

### Input: 
* .bgl.phased (You can convert plink file to .bgl.phased using Beagle 5)  (TODO: support for plink file if conditional analysis is not done)
* .fam (the same as plink)
* .phe (the same as plink)
* regular expression for identifying multialleic marker
* (optional) .covar (the same as plink)
* (optional) .condition (the same as plink except that the )
    
### Ouput:
* p-value for each merged marker (AA_A_1_Residue->AA_A_1) (HLA_A\*--:-- ->HLA_A)  chisquare - degree freedom - p-value
    


Functionality

* support HLA gene multialleic Omnibus test

Faster than R with same simulation settings and parameters
* manually convert categorial input to onehot-encoded input
* numpy use modules optimized for cpu(blas)

* Integrity check between .fam, pedigree information from .aa
* Simpler parameter parsing
* can configure CPU thread limit


https://stackoverflow.com/questions/48898712/very-slow-glm-logistic-regression-in-r

In [1]:
import os
import argparse
import logging
import time
import socket

import pandas as pd
import numpy as np
from scipy.stats import chi2

import statsmodels.api as sm

In [2]:
def dir_path(path):
    if os.path.exists(os.path.dirname(path)):
        return path
    else:
        raise argparse.ArgumentTypeError(f"readable_dir:{path} is not a valid path")

def file_path(path):
    if os.path.isfile(path):
        return path
    else:
        raise argparse.ArgumentTypeError(f"readable_dir:{path} is not a valid path")


parser = argparse.ArgumentParser(description='Omnibus test')

#required mode
parser.add_argument('--assoc',choices=['linear','logistic'],required=True)

#required output file
parser.add_argument('--out', type=dir_path,required=True,help='output file prefix. (prefix.log, prefix.assoc will be generated)')

#required input files
parser.add_argument('--fam', type=file_path,required=True,help='plink fam file')
parser.add_argument('--aa', type=file_path,required=True,help='AA call text file. For more information, refer to documentation. (like plink ped)')
parser.add_argument('--pheno', type=file_path,required=True,help='format is the same as plink. Tab-delimited file without header of which the first and second columns is family and within-family IDs respectively and the third columns are pheotype')

#optional
parser.add_argument('--covar', type=file_path,help='format is the same as plink')
parser.add_argument('--condition-list',type=file_path,help='format is the same as plink')

_StoreAction(option_strings=['--condition-list'], dest='condition_list', nargs=None, const=None, default=None, type=<function file_path at 0x7fc924154598>, choices=None, help='format is the same as plink', metavar=None)

In [3]:
debug=False
#debug=True

if debug:
    args=parser.parse_args('\
--assoc linear \
--out data/out_assoc/hba1c/step_02.omnibus \
--pheno data/out_assoc/hba1c/phenotype.phe \
--fam data/genotype/4_merge/KCHIP_HLA_SNP_1000G_merged.fam \
--covar data/out_assoc/hba1c/step_02.omnibus.covar.temp \
--aa data/out_assoc/hba1c/step_02.aa \
--condition-list data/out_assoc/hba1c/step_02.omnibus.cond\
'.split(' '))
else:
    args=parser.parse_args()

In [4]:
log = logging.getLogger('logger')
log.setLevel(logging.DEBUG)

log_file_path=args.out+'.log'
fileHandler = logging.FileHandler(log_file_path)
streamHandler = logging.StreamHandler()

formatter = logging.Formatter('%(message)s')
fileHandler.setFormatter(formatter)
streamHandler.setFormatter(formatter)

log.addHandler(fileHandler)
log.addHandler(streamHandler)

In [5]:
log.info_head=lambda x: log.info('-'*int((100-len(x))/2)+x+'-'*int((100-len(x))/2))

In [ ]:
log.info_head("**********************************************************")
log.info("* Amino Acid Association Test (Omnibus Test)")
log.info("* version 1.0")
log.info("* (C) 2020-, Seoul National University")
log.info("* Amino Acid Association Test (Omnibus Test)")
log.info("* Please report bugs to: Chanwoo Kim <ch6845@snu.ac.kr>")
log.info_head("**********************************************************")

In [9]:
log.info("Start time: "+time.strftime('%c', time.localtime(time.time())))

Start time: Sat Feb 29 20:39:53 2020


In [8]:
log.info('Working directory: '+os.getcwd())
log.info('Hostname: '+socket.gethostname())

Working directory: /data/ch6845/MHC_phewas_testbench


In [7]:
log.info('Parameters\n'+'\n'.join(['--{} {}'.format(key,value) for key,value in vars(args).items()]))

Parameters
--assoc linear
--out data/out_assoc/hba1c/step_02.omnibus
--fam data/genotype/4_merge/KCHIP_HLA_SNP_1000G_merged.fam
--aa data/out_assoc/hba1c/step_02.aa
--pheno data/out_assoc/hba1c/phenotype.phe
--covar data/out_assoc/hba1c/step_02.omnibus.covar.temp
--condition_list data/out_assoc/hba1c/step_02.omnibus.cond


In [10]:
assoc=args.assoc
out=args.out

In [11]:
log.info_head("Data Loading")

--------------------------------------------Data Loading--------------------------------------------


# parse input files

In [12]:
fam=pd.read_csv(args.fam,header=None,sep=' ',names=['FID','IID','fID','mID','sex','pheno']).astype({'FID':str,'IID':str,'fID':str,'mID':str,'sex':int,'pheno':float})

In [13]:
log.info("{} samples ({} males, {} females) loaded from {}".format(fam.shape[0],(fam['sex']==1).sum(),(fam['sex']==2).sum(),args.fam))

125673 samples (45647 males, 80026 females) loaded from data/genotype/4_merge/KCHIP_HLA_SNP_1000G_merged.fam


In [14]:
aa_IID_list=None
aa_sex_list=None
aa_marker_name_list=[]
aa_marker_data_list=[]
with open(args.aa,'r') as f:
    line_cnt=0
    while True:
        line=f.readline()            
        if not line:
            break        
        line_cnt+=1
        line_split=line.strip().split(' ')
        line_type,line_id,line_data=line_split[0],line_split[1],line_split[2:]
        if line_type=='P':
            pass
        elif line_type=='fID':
            pass
        elif line_type=='mID':
            pass        
        elif line_type=='I':        
            aa_IID_list1=np.array([line_data[i] for i in range(0,len(line_data),2)])
            aa_IID_list2=np.array([line_data[i+1] for i in range(0,len(line_data),2)])
            if np.all(aa_IID_list1==aa_IID_list2):
                aa_IID_list=aa_IID_list1
            else:
                raise
        elif line_type=='C':
            aa_sex_list1=np.array([line_data[i] for i in range(0,len(line_data),2)])
            aa_sex_list2=np.array([line_data[i+1] for i in range(0,len(line_data),2)])
            if np.all(aa_sex_list1==aa_sex_list2):
                aa_sex_list=aa_sex_list1.astype(int)
            else:
                raise
        elif line_type=='M':
            aa_marker_name_list.append(line_id)
            line_data=np.array([np.nan if x=='NA' else x for x in line_data])
            aa_marker_data_list.append(line_data)
        else:
            print(line_type)
            raise 
            
aa_marker_name_list_aaonly=pd.Series(aa_marker_name_list)[pd.Series(aa_marker_name_list).str.slice(stop=3)=='AA_'].values            

In [15]:
log.info("{} variants ({} AAs) loaded from {}".format(len(aa_marker_name_list),len(aa_marker_name_list_aaonly),args.aa))

382 variants (361 AAs) loaded from data/out_assoc/hba1c/step_02.aa


In [16]:
pheno=pd.read_csv(args.pheno,header=None,sep='\t',names=['FID','IID','pheno'])
pheno['pheno']=pheno['pheno'].replace(-9,np.nan)

In [17]:
log.info("pheotype loaded from {}".format(args.pheno))

pheotype loaded from data/out_assoc/hba1c/phenotype.phe


In [18]:
if args.assoc=='linear':
    assert len(pheno['pheno'].unique())>2
else:
    assert np.all(np.isnan(pheno['pheno'])|(pheno['pheno']==1)|(pheno['pheno']==2))
    pheno['pheno']=pheno['pheno']-1

In [19]:
log.info("{} pheotype loaded from {}".format(pheno.shape[0],args.pheno))
log.info("Among them, valid: {}, missing: {}".format((~pheno['pheno'].isnull()).sum(),pheno['pheno'].isnull().sum()))
if assoc=='linear':
    log.info("mean={:4f} std={:4f} median={:4f} min={:4f} max={:4f}".format(pheno['pheno'].mean(),pheno['pheno'].std(),pheno['pheno'].median(),pheno['pheno'].min(),pheno['pheno'].max()))
else:
    log.info("case: {} / control: {}".format((pheno['pheno']==1).sum(),(pheno['pheno']==0).sum()))

125673 pheotype loaded from data/out_assoc/hba1c/phenotype.phe
Among them, valid: 56650, missing: 69023
mean=5.644237 std=0.499977 median=5.600000 min=4.200000 max=8.000000


# parse optional input files

In [20]:
if args.covar is None:
    covar=fam.iloc[:,:2]
else:
    covar=pd.read_csv(args.covar,sep='\t')
    covar.columns=['FID','IID']+covar.columns[2:].tolist()
    covar=covar.astype({'FID':str,'IID':str})
    
    covar.iloc[:,2:]=covar.iloc[:,2:].astype(float)
    covar.iloc[:,2:]=covar.iloc[:,2:].replace(-9,np.nan)
    
    log.info("{} covariates loaded from {}".format(len(covar.columns[2:]),args.covar))

9 covariates loaded from data/out_assoc/hba1c/step_02.omnibus.covar.temp


In [21]:
if args.condition_list is None:
    condition_list=[]
else:
    with open(args.condition_list,'r') as f:
        condition_list=f.read().strip().split('\n')
        if condition_list[0]=='':
            condition_list=[]
            log.warning("Empty --condition-list {}".format(args.condition_list))
        else:
            log.info("{} conditions loaded from --condition-list {}".format(len(condition_list),args.condition_list))
            log.info(', '.join(condition_list))

77 conditions loaded from --condition-list data/out_assoc/hba1c/step_02.omnibus.cond
AA_C_-15, AA_C_-17, AA_C_-9, AA_C_1, AA_C_103, AA_C_11, AA_C_113, AA_C_114, AA_C_116, AA_C_138, AA_C_14, AA_C_147, AA_C_152, AA_C_156, AA_C_16, AA_C_163, AA_C_173, AA_C_175, AA_C_177, AA_C_184, AA_C_194, AA_C_21, AA_C_211, AA_C_219, AA_C_24, AA_C_248, AA_C_253, AA_C_261, AA_C_267, AA_C_273, AA_C_275, AA_C_285, AA_C_295, AA_C_303, AA_C_304, AA_C_305, AA_C_306, AA_C_307, AA_C_321, AA_C_324, AA_C_326, AA_C_339, AA_C_35, AA_C_49, AA_C_6, AA_C_66, AA_C_73, AA_C_77, AA_C_80, AA_C_9, AA_C_90, AA_C_91, AA_C_94, AA_C_95, AA_C_97, AA_C_99, HLA_C*01:02, HLA_C*02:02, HLA_C*03:02, HLA_C*03:03, HLA_C*03:04, HLA_C*04:01, HLA_C*05:01, HLA_C*06:02, HLA_C*07:02, HLA_C*07:04, HLA_C*07:06, HLA_C*08:01, HLA_C*08:02, HLA_C*08:03, HLA_C*08:22, HLA_C*12:02, HLA_C*12:03, HLA_C*14:02, HLA_C*14:03, HLA_C*15:02, HLA_C*15:05


In [22]:
#

# check idx integrity

In [23]:
assert np.all(fam['IID']==aa_IID_list)
assert np.all(fam['sex']==aa_sex_list)
assert len(aa_marker_name_list)==len(aa_marker_data_list)

assert np.all(fam['IID']==covar['IID'])
assert len(set(condition_list).difference(set(aa_marker_name_list)))==0

# Run regression

In [24]:
log.info_head("Checking missing values(observations)")

-------------------------------Checking missing values(observations)-------------------------------


In [25]:
def marker_data_to_onehot(aa_marker_data):
    
    aa_marker_data_unique=np.unique(aa_marker_data)
    aa_marker_data_unique_nonan=aa_marker_data_unique[aa_marker_data_unique!='nan'].tolist()
    aa_marker_data_unique_nan=aa_marker_data_unique_nonan+['nan']
    
    aa_marker_data_int_nan=list(map(lambda x: aa_marker_data_unique_nan.index(x),aa_marker_data))
    
    aa_marker_data_onehot_nan=np.zeros((len(aa_marker_data),len(aa_marker_data_unique_nan)))
    aa_marker_data_onehot_nan[np.arange(len(aa_marker_data)),aa_marker_data_int_nan]=1
    aa_marker_data_onehot_nonan=aa_marker_data_onehot_nan[:,:-1]
    
    return aa_marker_data_unique_nonan,aa_marker_data_onehot_nonan

def prepare_onehot(aa_marker_data_unique_nonan,aa_marker_data_onehot_nonan,set_nan=True,cut_mostfrequent=True):
    assert len(aa_marker_data_unique_nonan)==aa_marker_data_onehot_nonan.shape[1]
    assert np.isnan(aa_marker_data_onehot_nonan).sum()==0
    assert (np.nan not in aa_marker_data_unique_nonan) and ('nan' not in aa_marker_data_unique_nonan)
    
    aa_marker_data_onehot_nonan_sumrow=np.sum(aa_marker_data_onehot_nonan,axis=1)
    aa_marker_data_onehot_nonan_sumcol=np.sum(aa_marker_data_onehot_nonan,axis=0)
    #print(aa_marker_data_onehot_nonan_sumcol,np.argmax(aa_marker_data_onehot_nonan_sumcol))
    #print(aa_marker_data_onehot_nonan_sumrow.shape,aa_marker_data_onehot_nonan_sumcol.shape)
    #print(np.argmax(aa_marker_data_onehot_nonan_sumcol))
    if set_nan:
        aa_marker_data_onehot_nonan[aa_marker_data_onehot_nonan_sumrow!=1,:]=np.nan
    if cut_mostfrequent:
        aa_marker_data_unique_nonan=np.delete(aa_marker_data_unique_nonan, np.argmax(aa_marker_data_onehot_nonan_sumcol))
        aa_marker_data_onehot_nonan=np.delete(aa_marker_data_onehot_nonan, np.argmax(aa_marker_data_onehot_nonan_sumcol), axis=1)
    return aa_marker_data_unique_nonan,aa_marker_data_onehot_nonan

## y data

In [26]:
y_data_pheno=np.repeat(pheno['pheno'].values,2)

In [27]:
y_data_pheno_nan=np.isnan(y_data_pheno)

In [28]:
if np.var(y_data_pheno[~y_data_pheno_nan])==0:
    log.error("No variance in y_data")
    raise

In [29]:
log.info("missing values in y_data_pheno: {}".format(y_data_pheno_nan.sum()))

missing values in y_data_pheno: 138046


## x data

In [30]:
x_data_intercept=np.array([np.ones(2*fam.shape[0])]).transpose()

In [31]:
x_data_covar=np.repeat(covar.iloc[:,2:].values,2,axis=0)

In [32]:
x_data_covar_nan=np.any(np.isnan(x_data_covar),axis=1)
log.info("missing values in x_data_covar: {}".format(x_data_covar_nan.sum()))

missing values in x_data_covar: 0


In [33]:
x_data_covar_varcheck=np.array([np.var(covar) for covar in x_data_covar[~x_data_covar_nan].transpose()])==0

if np.any(x_data_covar_varcheck):
    log.info("Removed covariates of no variance"+", ".join(covar.columns[2:][x_data_covar_varcheck].tolist()))
    x_data_covar=np.delete(x_data_covar,np.arange(len(x_data_covar_varcheck))[x_data_covar_varcheck],axis=1)

In [34]:
if (x_data_covar.size!=0) and (np.linalg.matrix_rank(x_data_covar)<x_data_covar.shape[1]):
    log.info("duplicated covariates were found. (rank(covariates)< # of covarirates))")
    #raise
    #x_data_covar=np.delete(x_data_covar,np.arange(len(x_data_covar_varcheck))[x_data_covar_varcheck],axis=1)

In [35]:
temp_list=[x_data_intercept[:,0:0]]
for aa_marker_name in condition_list:
    aa_marker_data=aa_marker_data_list[aa_marker_name_list.index(aa_marker_name)]

    aa_marker_data_unique,aa_marker_data_onehot=marker_data_to_onehot(aa_marker_data)
    aa_marker_data_unique_cut,aa_marker_data_onehot_cut=prepare_onehot(aa_marker_data_unique,aa_marker_data_onehot,set_nan=True,cut_mostfrequent=True)        
    temp_list.append(aa_marker_data_onehot_cut)
    
x_data_condition=np.concatenate(temp_list,axis=1)
x_data_condition_nan=np.any(np.isnan(x_data_condition),axis=1)  

In [36]:
log.info("missing values in x_data_condition: {}".format(x_data_condition_nan.sum()))

missing values in x_data_condition: 97


In [37]:
"""
x_data_condition=np.delete(x_data_condition,list(todel),axis=1)
todel=set()
for i in range(x_data_condition.shape[1]):
    for j in range(i+1,x_data_condition.shape[1]):
        if np.corrcoef(x_data_condition[~x_data_condition_nan][:,i],x_data_condition[~x_data_condition_nan][:,j])[1,0]>0.9:
            todel.add(i)
            todel.add(j)
x_data_condition_nan=np.any(np.isnan(x_data_condition),axis=1)  
np.linalg.matrix_rank(x_data_condition[~x_data_condition_nan]),x_data_condition.shape[1]
"""            

'\nx_data_condition=np.delete(x_data_condition,list(todel),axis=1)\ntodel=set()\nfor i in range(x_data_condition.shape[1]):\n    for j in range(i+1,x_data_condition.shape[1]):\n        if np.corrcoef(x_data_condition[~x_data_condition_nan][:,i],x_data_condition[~x_data_condition_nan][:,j])[1,0]>0.9:\n            todel.add(i)\n            todel.add(j)\nx_data_condition_nan=np.any(np.isnan(x_data_condition),axis=1)  \nnp.linalg.matrix_rank(x_data_condition[~x_data_condition_nan]),x_data_condition.shape[1]\n'

In [38]:
log.info_head("Regression")

---------------------------------------------Regression---------------------------------------------


In [ ]:
log.info('[{:3d}/{:3d}] {:10s} {:15s} {:5s} {:.5s}({}) {:.5s}'.format(
                            0,
                            len(aa_marker_name_list_aaonly),
                            'ID',
                            'residues',
                            'n_obs',
                            'chisq',
                            'df',
                            'P'
                        ))
assoc_result_list=[]

for idx,aa_marker_name in enumerate(aa_marker_name_list_aaonly):
    aa_marker_data=aa_marker_data_list[idx]
    aa_marker_data_unique,aa_marker_data_onehot=marker_data_to_onehot(aa_marker_data)
    aa_marker_data_unique_cut,aa_marker_data_onehot_cut=prepare_onehot(aa_marker_data_unique,aa_marker_data_onehot,set_nan=True,cut_mostfrequent=True)
    
    x_data_aa_marker=aa_marker_data_onehot_cut
    x_data_aa_marker_nan=np.any(np.isnan(x_data_aa_marker),axis=1)
    
    x_data_nan=np.logical_or.reduce([x_data_covar_nan,x_data_condition_nan,x_data_aa_marker_nan])
    
    y_data=y_data_pheno
    y_data_nan=y_data_pheno_nan    
    x_y_data_nan=(x_data_nan)|(y_data_nan)


    x_data_null=np.concatenate([x_data_intercept,x_data_covar,x_data_condition,x_data_aa_marker],axis=1)[~x_y_data_nan]
    x_data_alt=np.concatenate([x_data_intercept,x_data_covar,x_data_condition],axis=1)[~x_y_data_nan]
    y_data=y_data[~x_y_data_nan]
    
    
    family=(sm.families.Gaussian() if assoc=='linear' else sm.families.Binomial())
    model_null = sm.GLM(y_data,x_data_null, family=family,missing='raise')
    model_alt = sm.GLM(y_data,x_data_alt, family=family,missing='raise')
    
    try:
        result_alt = model_alt.fit()
        result_null = model_null.fit()
    except sm.tools.sm_exceptions.PerfectSeparationError as e:
        nobs=np.nan
        chisq_diff=np.nan
        df_diff=np.nan
        p_value=np.nan
    else:
        assert result_alt.nobs==result_null.nobs
        nobs=result_alt.nobs
        chisq_diff=2*(result_null.llf-result_alt.llf)
        df_diff=result_null.df_model-result_alt.df_model
        p_value=chi2.sf(chisq_diff,df_diff)        
        #p_value=1 - chi2.cdf(chisq_diff,df_diff)

    #print(result_alt.summary())
    assoc_result={'idx':idx+1,
                  'ID':aa_marker_name,
                  'residues':','.join(aa_marker_data_unique),
                  'n_obs':nobs,
                  'chisq':chisq_diff,
                  'df':df_diff,
                'P':p_value}
    
    
    assoc_result_list.append(assoc_result)
    
    log.info('[{:3d}/{:3d}] {:10s} {:15s} {:5f} {:.5f}({}) {:e}'.format(
                                assoc_result['idx'],
                                len(aa_marker_name_list_aaonly),
                                assoc_result['ID'],
                                assoc_result['residues'],
                                assoc_result['n_obs'],
                                assoc_result['chisq'],
                                assoc_result['df'],
                                assoc_result['P']
                            ))


In [ ]:
pd.DataFrame(assoc_result_list)[['idx','ID','residues','n_obs','chisq','df','P']].to_csv(out+'.'+assoc,sep='\t',index=None)

In [9]:
log.info("End time: "+time.strftime('%c', time.localtime(time.time())))

Start time: Sat Feb 29 20:39:53 2020
